*Note:  This notebooks should be executed against a Serverelss SQL Warehouse.*

# Interacting with the FHIR Bundles Entry Elements to Parse Resources 
***

## Notebook Setup
***

In [0]:
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
USE IDENTIFIER("fhir_workshop." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

## FHIR Bundle Meta 
***

Depending on the source of the FHIR bundle there may be a full set of metadata about the bundle itself.  This is standard practice for providers that clean and standardize your incoming FHIR data, or convert your C-CDAs, ADTs, or other clinical data to the FHIR format such as Redox.  Note that Synthea doesn't include a bundle meta, therefore we're not going to fully parse the Meta object quite as much as we would if it were available.  We do have one Redox JSON loaded into our bronze tables that we can at least review.  Normally we'd split the Meta object object into separate columns exactly like we would for the Bundle's "entry resources" but that would be overkill for this class. 

What is helpful however is to have a place for items such as the `file_metadata` and then `ingest_time` avaialble in a silver table so that we don't need to continue to bring that along for the ride with the Resource tables.  

In [0]:
DROP TABLE IF EXISTS fhir_bundle_metadata;

In [0]:
CREATE OR REFRESH STREAMING TABLE fhir_bundle_metadata 
COMMENT 'Original FHIR Bundle Metadata'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  bundle_uuid
  ,file_metadata
  ,ingest_time
FROM STREAM (fhir_bronze_variant);

In [0]:
DROP TABLE IF EXISTS fhir_resources;

In [0]:
CREATE OR REFRESH STREAMING TABLE fhir_resources 
COMMENT 'Exploded FHIR Resources'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  bundle_uuid
  ,CAST(entry.value:fullUrl AS STRING) as fullUrl
  ,CAST(entry.value:resource.resourceType AS STRING) as resourceType
  ,resource.*
FROM 
  STREAM(fhir_bronze_variant)
  ,LATERAL variant_explode(fhir:entry) as entry
  ,LATERAL variant_explode(entry.value:resource) as resource

In [0]:
SELECT * FROM fhir_resources;

In [0]:
DROP TABLE IF EXISTS fhir_resource_schemas;

In [0]:
CREATE OR REFRESH STREAMING TABLE fhir_resource_schemas 
COMMENT 'Schemas of inferred from FHIR Resource Variant Data Types'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  resourceType
  ,key as resource
  ,schema_of_variant_agg(value) as schema_of_variant
  ,REPLACE(schema_of_variant_agg(value), 'OBJECT', 'STRUCT') as schema_as_struct
FROM STREAM(fhir_resources)
GROUP BY 
  resourceType
  ,key

In [0]:
select * from fhir_resource_schemas;